In [ ]:
from astropy.time import Time
import numpy as np
import pandas as pd

import df_utils
from lsst_efd_client import EfdClient

In [ ]:
#location = "tucson"
location = "summit"

In [ ]:
client = EfdClient(location)

In [ ]:
#csc = "ATMonochromator"
#csc = "Environment"
csc = "ATArchiver"
#csc = "ATMCS"
index = None
d1 = Time("2020-01-27", scale='utc')
d2 = Time("2020-01-31", scale='utc')
summary_state = 3  # FAULT
time_window = 10.0  # seconds
time_format = '%Y-%m-%dT%H:%M:%S.%f'

In [ ]:
ss_df = await client.select_time_series(f"lsst.sal.{csc}.logevent_summaryState", 
                                        ["private_sndStamp",
                                         "private_rcvStamp",
                                         "summaryState"], d1, d2, index=index)

In [ ]:
try:
    ss_df = df_utils.convert_timestamps(ss_df, ["private_sndStamp",
                                                "private_rcvStamp"])
except KeyError:
    print("NO DATA FOUND!!!!")

In [ ]:
ec_df = await client.select_time_series(f"lsst.sal.{csc}.logevent_errorCode",
                                        ["private_sndStamp",
                                         "private_rcvStamp",
                                         "errorCode",
                                         "errorReport",
                                         "traceback"], d1, d2)
ec_df = df_utils.convert_timestamps(ec_df, ["private_sndStamp",
                                            "private_rcvStamp"])

In [ ]:
timedelta = pd.Timedelta(time_window, unit='sec')

In [ ]:
ss_df = ss_df[ss_df.summaryState == summary_state]

In [ ]:
if ss_df.private_sndStamp_date.size:
    for timestamp in ss_df.private_sndStamp_date:
        print("######################################")
        print(f"Fault Summary State: {timestamp.strftime(time_format)}")
        t1 = timestamp - timedelta
        t2 = timestamp + timedelta
        error_indexes = ec_df.private_sndStamp_date.to_numpy()
        index = np.logical_and(error_indexes > t1, error_indexes < t2)
        error = ec_df[index]
        if error.shape[0] > 1:
            print("Multiple errorCode messages associated with Fault State!")
        for entry in range(error.shape[0]):
            print("----------------------------------")
            snd_stamp = error.private_sndStamp_date[entry].strftime(time_format)
            rcv_stamp = error.private_rcvStamp_date[entry].strftime(time_format)
            print(f"Fault State Report: {snd_stamp} ({rcv_stamp})")
            print(f"Error Code: {error.errorCode.values[entry]}")
            print(f"Error Report: {error.errorReport.values[entry]}")
            print(f"Traceback: {error.traceback.values[entry]}")
else:
    print(f"No fault states found for {csc}")